# 로컬에서 훈련 하기
- https://www.kaggle.com/code/mitanshuchakrawarty/fine-tune-llm-for-text-summary

## 1. 환경 셋업

In [1]:
# !huggingface-cli login --token ""

In [15]:
import os 
os.environ['TRANSFORMERS_CACHE'] = "/home/ec2-user/SageMaker/.cache" 
os.environ['HF_DATASETS_CACHE'] = "/home/ec2-user/SageMaker/.cache" 
os.environ['HF_HOME'] = "/home/ec2-user/SageMaker/.cache"

In [16]:
%store -r data_folder
%store -r train_data_json 
%store -r validation_data_json 
%store -r test_data_json 

print("data_folder: ", data_folder)
print("train_data_json: ", train_data_json)
print("validation_data_json: ", validation_data_json)
print("test_data_json: ", test_data_json)

data_folder:  ../data/naver-news-summarization-ko
train_data_json:  ../data/naver-news-summarization-ko/train_dataset.json
validation_data_json:  ../data/naver-news-summarization-ko/validation_dataset.json
test_data_json:  ../data/naver-news-summarization-ko/test_dataset.json


In [17]:
import torch
import time
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from datasets import Dataset, load_dataset
from datasets import load_dataset, load_metric
from transformers import pipeline, set_seed
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import warnings
warnings.filterwarnings("ignore")

## 2. 베이스 모델 준비

In [18]:
model_id = "meta-llama/Meta-Llama-3-8B"
output_dir = "/home/ec2-user/SageMaker/models/llama-3-8b-naver-news"

### Config YAML 파일 생성

In [19]:
%%writefile llama_3_8b_fsdp_qlora.yaml
# script parameters
model_id:  "meta-llama/Meta-Llama-3-8B" # Hugging Face model id
dataset_path: "../data/naver-news-summarization-ko"                      # path to dataset
max_seq_len:  2048              # max sequence length for model and packing of the dataset
# training parameters
output_dir: "/home/ec2-user/SageMaker/models/llama-3-8b-naver-news" # Temporary output directory for model checkpoints
report_to: "tensorboard"               # report metrics to tensorboard
learning_rate: 0.0002                  # learning rate 2e-4
lr_scheduler_type: "constant"          # learning rate scheduler
num_train_epochs: 1                    # number of training epochs
per_device_train_batch_size: 1         # batch size per device during training
per_device_eval_batch_size: 1          # batch size for evaluation
gradient_accumulation_steps: 2         # number of steps before performing a backward/update pass
optim: adamw_torch                     # use torch adamw optimizer
logging_steps: 10                      # log every 10 steps
save_strategy: epoch                   # save checkpoint every epoch
evaluation_strategy: epoch             # evaluate every epoch
max_grad_norm: 0.3                     # max gradient norm
warmup_ratio: 0.03                     # warmup ratio
bf16: true                             # use bfloat16 precision
tf32: true                             # use tf32 precision
gradient_checkpointing: true           # use gradient checkpointing to save memory
# FSDP parameters: https://huggingface.co/docs/transformers/main/en/fsdp
fsdp: "full_shard auto_wrap offload" # remove offload if enough GPU memory
fsdp_config:
  backward_prefetch: "backward_pre"
  forward_prefetch: "false"
  use_orig_params: "false"

Overwriting llama_3_8b_fsdp_qlora.yaml


## 3. 훈련 Script 실행

In [20]:
!ACCELERATE_USE_FSDP=1 FSDP_CPU_RAM_EFFICIENT_LOADING=1 torchrun --nproc_per_node=4 \
../scripts/run_fsdp_qlora.py \
--config llama_3_8b_fsdp_qlora.yaml

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[2024-06-27 14:04:27,796] torch.distributed.run: [WARNING] 
[2024-06-27 14:04:27,796] torch.distributed.run: [WARNING] *****************************************
[2024-06-27 14:04:27,796] torch.distributed.run: [WARNING] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
[2024-06-27 14:04:27,796] torch.distributed.run: [WARNING] *****************************************
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  

## 4. 베이스 모델과 훈련된 모델 머지

In [21]:
model_id, output_dir

('meta-llama/Meta-Llama-3-8B',
 '/home/ec2-user/SageMaker/models/llama-3-8b-naver-news')

### 모델 머지 및 로컬에 저장

In [22]:
from peft import AutoPeftModelForCausalLM
import torch

# Load PEFT model on CPU

model = AutoPeftModelForCausalLM.from_pretrained(
    output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)  
# Merge LoRA and base model and save
merged_model = model.merge_and_unload()
merged_model.save_pretrained(output_dir,safe_serialization=True, max_shard_size="2GB")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### 머지된 모델 로딩

In [23]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer 


# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  pretrained_model_name_or_path = output_dir,
  torch_dtype=torch.float16,
  quantization_config= {"load_in_4bit": True},
  device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 5. 추론

### 테스트 데이터 셋 로딩

In [24]:
test_data_json

'../data/naver-news-summarization-ko/test_dataset.json'

In [25]:
from datasets import load_dataset 
from random import randint

# Load our test dataset
eval_dataset = load_dataset("json", data_files=test_data_json, split="train")
rand_idx = randint(0, len(eval_dataset))
messages = eval_dataset[rand_idx]["messages"][:2]
rand_idx, messages

(1,
 [{'content': 'You are an AI assistant specialized in news articles.Your role is to provide accurate summaries and insights in Korean. Please analyze the given text and provide concise, informative summaries that highlight the key goals and findings.',
   'role': 'system'},
  {'content': 'Please summarize the goals for journalist in this text:\n\nLG생활건강 글로벌에코리더 YOUTH가 3일 강원 동해시 망상해변에서 비치코밍 캠페인을 진행했다. LG생활건강 제공 LG생활건강이 ESG 환경·사회·지배구조 경영 일환으로 청년 환경 활동가들과 바다 쓰레기 청소 활동을 벌였다. LG생활건강은 MZ세대 밀레니얼 Z세대 기후환경 활동가 집단 ‘글로벌에코리더 YOUTH’ 100여 명과 지난 3일 강원 동해 망상해변에서 비치코밍 캠페인을 펼쳤다고 4일 발표했다. 비치코밍이란 해안가에 버려지거나 떠밀려온 플라스틱 유리 조각 등을 빗질하듯이 주워 모으는 것을 의미한다. LG생활건강은 환경단체 에코맘코리아와 함께 올초부터 환경에 관심이 많은 20세 이상 청년 100여 명을 선발해 MZ세대 기후환경 활동가로 육성하는 글로벌에코리더 YOUTH 사업을 이어오고 있다. 이들은 4 7명이 한조가 돼 대학 축제에서 홍보 부스를 운영하고 길거리 쓰레기를 줍는 등 대학가와 지역사회를 중심으로 가시적인 변화를 끌어내기 위해 노력하고 있다. 이번 행사는 유엔환경계획 UNEP 이 일회용 플라스틱으로 인한 해양오염을 줄이기 위해 세계 청년들과 함께하는 ‘플라스틱 타이드 터너스 챌린지’ 캠페인의 일환으로 진행됐다. 시민 스쿠버다이버 원주환경청 동해시 관계자 등도 참여해 해양 정화 활동을 펼쳤다. 회사 측은 전문가들이 해양쓰레기

### 추론

In [26]:
# Test on sample 
input_ids = tokenizer.apply_chat_template(messages,add_generation_prompt=True,return_tensors="pt").to(model.device)
outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id= tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]

print(f"**Query:**\n{eval_dataset[rand_idx]['messages'][1]['content']}\n")
print(f"**Original Answer:**\n{eval_dataset[rand_idx]['messages'][2]['content']}\n")
print(f"**Generated Answer:**\n{tokenizer.decode(response,skip_special_tokens=True)}")



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


**Query:**
Please summarize the goals for journalist in this text:

LG생활건강 글로벌에코리더 YOUTH가 3일 강원 동해시 망상해변에서 비치코밍 캠페인을 진행했다. LG생활건강 제공 LG생활건강이 ESG 환경·사회·지배구조 경영 일환으로 청년 환경 활동가들과 바다 쓰레기 청소 활동을 벌였다. LG생활건강은 MZ세대 밀레니얼 Z세대 기후환경 활동가 집단 ‘글로벌에코리더 YOUTH’ 100여 명과 지난 3일 강원 동해 망상해변에서 비치코밍 캠페인을 펼쳤다고 4일 발표했다. 비치코밍이란 해안가에 버려지거나 떠밀려온 플라스틱 유리 조각 등을 빗질하듯이 주워 모으는 것을 의미한다. LG생활건강은 환경단체 에코맘코리아와 함께 올초부터 환경에 관심이 많은 20세 이상 청년 100여 명을 선발해 MZ세대 기후환경 활동가로 육성하는 글로벌에코리더 YOUTH 사업을 이어오고 있다. 이들은 4 7명이 한조가 돼 대학 축제에서 홍보 부스를 운영하고 길거리 쓰레기를 줍는 등 대학가와 지역사회를 중심으로 가시적인 변화를 끌어내기 위해 노력하고 있다. 이번 행사는 유엔환경계획 UNEP 이 일회용 플라스틱으로 인한 해양오염을 줄이기 위해 세계 청년들과 함께하는 ‘플라스틱 타이드 터너스 챌린지’ 캠페인의 일환으로 진행됐다. 시민 스쿠버다이버 원주환경청 동해시 관계자 등도 참여해 해양 정화 활동을 펼쳤다. 회사 측은 전문가들이 해양쓰레기의 심각성과 문제 해결에 관해 토론하는 시간도 마련했다고 설명했다. LG생활건강 관계자는 “캠페인에 참여한 MZ세대가 기후 위기와 해양쓰레기 문제에 더 많은 관심을 갖고 환경 활동가로 성장하도록 지원할 것”이라며 “청년들의 목소리를 경청하는 ESG 경영을 해나가겠다”고 밝혔다.

**Original Answer:**
LG생활건강은 환경단체 에코맘코리아와 함께 올초부터 환경에 관심이 많은 20세 이상 청년 100여 명을 선발해 MZ세대 기후환경 활동가로 육성하는 ‘플라스틱 타이드 터너스 챌린지’ 캠페인의 일